In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
import datetime as dat
import matplotlib as plt
import datetime as dt
import xgboost as xgb
from collections import Counter
%matplotlib inline
sns.set()

In [2]:
features_ventana1 = pd.read_csv('../xgb/features_auctions_lucas_ventana1.csv',low_memory=False).set_index('ref_hash')
aux_ventana1 = pd.read_csv('../xgb/features_events_lucas_ventana1.csv').set_index('ref_hash')
features_ventana1=features_ventana1.join(aux_ventana1, how = 'outer', lsuffix = '_auctions', rsuffix = '_events')
aux_ventana1 = pd.read_csv('../xgb/features_installs_lucas_ventana1.csv').set_index('ref_hash')
features_ventana1=features_ventana1.join(aux_ventana1, how = 'outer', rsuffix = '_installs')
aux_ventana1 = pd.read_csv('../xgb/features_clicks_lucas_ventana1.csv').set_index('ref_hash')
features_ventana1=features_ventana1.join(aux_ventana1, how = 'outer', rsuffix = '_clicks')
features_ventana1.columns
features_ventana1[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']]=features_ventana1[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']].fillna(259200)#3 dias pasado a segundos
#features_ventana1 = features_ventana1.fillna(0)

labels_st_ventana1 = pd.read_csv('../xgb/labels_st_ventana1.csv').set_index('device_id')
ventana1 = features_ventana1.join(labels_st_ventana1, how = 'left')#.fillna(259200)#3 dias pasado a segundos
#ventana1['tiempo_hasta_aparicion']=ventana1['tiempo_hasta_aparicion'].fillna(259200)#3 dias pasado a segundos
features_ventana1.head()

ventana1_no_conversion=ventana1.loc[ventana1['tiempo_hasta_aparicion'].isna()].copy()#personas que no convirtieron en esa ventana
ventana1_conversion=ventana1.loc[np.logical_not(ventana1['tiempo_hasta_aparicion'].isna())].copy()

In [3]:
tamanio_muestra_ventana1=ventana1_no_conversion.shape[0]
ventana1_no_conversion['tiempo_hasta_aparicion']=ventana1_no_conversion['tiempo_hasta_aparicion'].fillna(259200)#3 dias pasado a segundos
ventana1_no_conversion=ventana1_no_conversion.sample(n=int(tamanio_muestra_ventana1*0.5), replace=True, random_state=662)
ventana1=ventana1_conversion.append(ventana1_no_conversion)

In [4]:
ventana1_no_conversion.head()

,hour_sum_auctions,hour_mean_auctions,hour_std_auctions,hour_min_auctions,hour_max_auctions,tiempo_desde_inicio_de_ventana_sum,tiempo_desde_inicio_de_ventana_mean,tiempo_desde_inicio_de_ventana_std,tiempo_desde_inicio_de_ventana_min,tiempo_desde_inicio_de_ventana_max,...,hour_mean_clicks,hour_std_clicks,hour_min_clicks,hour_max_clicks,repeticiones_sum_clicks,repeticiones_mean_clicks,repeticiones_std_clicks,repeticiones_min_clicks,repeticiones_max_clicks,tiempo_hasta_aparicion
ref_hash,,,,,,,,,,,,,,,,,,,,,
5952340046936366939,NaN,NaN,NaN,NaN,NaN,259200.000000,NaN,259200.000000,259200.000000,259200.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0
5939582437857040523,NaN,NaN,NaN,NaN,NaN,259200.000000,NaN,259200.000000,259200.000000,259200.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0
1528199273078817943,NaN,NaN,NaN,NaN,NaN,259200.000000,NaN,259200.000000,259200.000000,259200.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0
7751235665132755718,202676.0,67558.666667,185.99552,67347.0,67696.0,202677.121108,67559.040369,185.725811,67347.721853,67696.360968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0
9156601139968186682,200431.0,66810.333333,16610.06184,56616.0,85977.0,546031.533280,182010.511093,83169.582399,85977.142413,230638.351249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259200.0


In [5]:
#pd.read_csv('../xgb/labels_st_ventana1.csv')

In [6]:
features_ventana2 = pd.read_csv('../xgb/features_auctions_lucas_ventana2.csv',low_memory=False).set_index('ref_hash')
aux_ventana2 = pd.read_csv('../xgb/features_events_lucas_ventana2.csv').set_index('ref_hash')
features_ventana2=features_ventana2.join(aux_ventana2, how = 'outer', lsuffix = '_auctions', rsuffix = '_events')
aux_ventana2 = pd.read_csv('../xgb/features_installs_lucas_ventana2.csv').set_index('ref_hash')
features_ventana2=features_ventana2.join(aux_ventana2, how = 'outer', rsuffix = '_installs')
aux_ventana2 = pd.read_csv('../xgb/features_clicks_lucas_ventana2.csv').set_index('ref_hash')
features_ventana2=features_ventana2.join(aux_ventana2, how = 'left', rsuffix = '_clicks')


features_ventana2[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']]=features_ventana2[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']].fillna(259200)#3 dias pasado a segundos
#features_ventana2 = features_ventana2.fillna(0)

labels_st_ventana2 = pd.read_csv('../xgb/labels_st_ventana2.csv').set_index('device_id')
ventana2 = features_ventana2.join(labels_st_ventana2, how = 'left')#.fillna(259200)#3 dias pasado a segundos
#ventana2['tiempo_hasta_aparicion']=ventana2['tiempo_hasta_aparicion'].fillna(259200)

ventana2_no_conversion=ventana2.loc[ventana2['tiempo_hasta_aparicion'].isna()].copy()#personas que no convirtieron en esa ventana
ventana2_conversion=ventana2.loc[np.logical_not(ventana2['tiempo_hasta_aparicion'].isna())].copy()

tamanio_muestra_ventana2=ventana2_conversion.shape[0]
ventana2_no_conversion['tiempo_hasta_aparicion']=ventana2_no_conversion['tiempo_hasta_aparicion'].fillna(259200)#3 dias pasado a segundos
ventana2_no_conversion=ventana2_no_conversion.sample(n=int(tamanio_muestra_ventana2*0.5), replace=True, random_state=662)
ventana2=ventana2_conversion.append(ventana2_no_conversion)


In [7]:
features_ventana3 = pd.read_csv('../xgb/features_auctions_lucas_ventana3.csv',low_memory=False).set_index('ref_hash')
aux_ventana3 = pd.read_csv('../xgb/features_events_lucas_ventana3.csv').set_index('ref_hash')
features_ventana3=features_ventana3.join(aux_ventana3, how = 'outer', lsuffix = '_auctions', rsuffix = '_events')
aux_ventana3 = pd.read_csv('../xgb/features_installs_lucas_ventana3.csv').set_index('ref_hash')
features_ventana3=features_ventana3.join(aux_ventana3, how = 'outer', rsuffix = '_installs')
aux_ventana3 = pd.read_csv('../xgb/features_clicks_lucas_ventana3.csv').set_index('ref_hash')
features_ventana3=features_ventana3.join(aux_ventana3, how = 'outer', rsuffix = '_clicks')


features_ventana3.columns
features_ventana3[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']]=features_ventana3[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']].fillna(259200)#3 dias pasado a segundos
#features_ventana3 = features_ventana3.fillna(0)

labels_st_ventana3 = pd.read_csv('../xgb/labels_st_ventana3.csv').set_index('device_id')
ventana3 = features_ventana3.join(labels_st_ventana3, how = 'left')#.fillna(259200)#3 dias pasado a segundos
#ventana3['tiempo_hasta_aparicion']=ventana3['tiempo_hasta_aparicion'].fillna(259200)


ventana3_no_conversion=ventana3.loc[ventana3['tiempo_hasta_aparicion'].isna()].copy()#personas que no convirtieron en esa ventana
ventana3_conversion=ventana3.loc[np.logical_not(ventana3['tiempo_hasta_aparicion'].isna())].copy()

tamanio_muestra_ventana3=ventana3_conversion.shape[0]
ventana3_no_conversion['tiempo_hasta_aparicion']=ventana3_no_conversion['tiempo_hasta_aparicion'].fillna(259200)#3 dias pasado a segundos
ventana3_no_conversion=ventana3_no_conversion.sample(n=int(tamanio_muestra_ventana3*0.5), replace=True, random_state=662)
ventana3=ventana3_conversion.append(ventana3_no_conversion)

In [16]:
features_ventana4 = pd.read_csv('../xgb/features_auctions_lucas_ventana4.csv',low_memory=False).set_index('ref_hash')
aux_ventana4 = pd.read_csv('../xgb/features_events_lucas_ventana4.csv').set_index('ref_hash')
features_ventana4=features_ventana4.join(aux_ventana4, how = 'outer', lsuffix = '_auctions', rsuffix = '_events')
aux_ventana4 = pd.read_csv('../xgb/features_installs_lucas_ventana4.csv').set_index('ref_hash')
features_ventana4=features_ventana4.join(aux_ventana4, how = 'outer', rsuffix = '_installs')
aux_ventana4 = pd.read_csv('../xgb/features_clicks_lucas_ventana4.csv').set_index('ref_hash')
features_ventana4=features_ventana4.join(aux_ventana4, how = 'outer', rsuffix = '_clicks')


features_ventana4[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']]=features_ventana4[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']].fillna(259200)#3 dias pasado a segundos

#features_ventana4 = features_ventana4.fillna(0)

labels_st_ventana4 = pd.read_csv('../xgb/labels_st_ventana4.csv').set_index('device_id')
ventana4 = features_ventana4.join(labels_st_ventana4, how = 'left')#.fillna(259200)#3 dias pasado a segundos

ventana4_no_conversion=ventana4.loc[ventana4['tiempo_hasta_aparicion'].isna()].copy()#personas que no convirtieron en esa ventana
ventana4_conversion=ventana4.loc[np.logical_not(ventana4['tiempo_hasta_aparicion'].isna())].copy()
ventana4_no_conversion['tiempo_hasta_aparicion']=ventana4_no_conversion['tiempo_hasta_aparicion']#.fillna(259200)#3 dias pasado a segundos
ventana4_no_conversion=ventana4_no_conversion.sample(frac=0.1, replace=True, random_state=662)
ventana4=ventana4_conversion.append(ventana4_no_conversion)




ventana4_no_conversion=ventana4.loc[ventana4['tiempo_hasta_aparicion'].isna()].copy()#personas que no convirtieron en esa ventana
ventana4_conversion=ventana4.loc[np.logical_not(ventana4['tiempo_hasta_aparicion'].isna())].copy()

tamanio_muestra_ventana4=ventana4_conversion.shape[0]
tamanio_muestra_ventana4
ventana4_no_conversion['tiempo_hasta_aparicion']=ventana4_no_conversion['tiempo_hasta_aparicion'].fillna(259200)#3 dias pasado a segundos
ventana4_no_conversion=ventana4_no_conversion.sample(n=int(tamanio_muestra_ventana4*0.1+1), replace=True, random_state=662)
ventana4=ventana4_conversion.append(ventana4_no_conversion)
ventana4_no_conversion.shape[0]

25792

In [17]:
#matriz = ventana1.append(ventana4,sort=False).reset_index('ref_hash')
matriz = ventana1.append(ventana2,sort=False).append(ventana3,sort=False).append(ventana4,sort=False).reset_index('ref_hash')
matriz=matriz.drop(['attributed_min','attributed_max'],axis=1)
#matriz.columns

In [18]:
matriz.head()
mean_prediction=matriz['tiempo_hasta_aparicion'].mean()
mean_prediction

119640.31363150368

In [19]:
filas = matriz.shape[0]
matriz.head()


,ref_hash,hour_sum_auctions,hour_mean_auctions,hour_std_auctions,hour_min_auctions,hour_max_auctions,tiempo_desde_inicio_de_ventana_sum,tiempo_desde_inicio_de_ventana_mean,tiempo_desde_inicio_de_ventana_std,tiempo_desde_inicio_de_ventana_min,...,hour_mean_clicks,hour_std_clicks,hour_min_clicks,hour_max_clicks,repeticiones_sum_clicks,repeticiones_mean_clicks,repeticiones_std_clicks,repeticiones_min_clicks,repeticiones_max_clicks,tiempo_hasta_aparicion
0,186034136943920,262546.0,37506.571429,32711.731111,11871.0,84877.0,1.126550e+06,160935.721167,70802.599949,60166.331894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.851558
1,345999128501141,2557316.0,53277.416667,9221.332601,9576.0,58033.0,2.902939e+06,60477.901544,26439.934566,54290.380240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164703.549846
2,360710529886978,879313.0,67639.461538,11518.707115,51449.0,80391.0,1.484119e+06,114163.007774,68866.738196,58572.637969,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245538.096215
3,416301579449694,1166126.0,33317.885714,29254.773979,4460.0,86349.0,6.350143e+06,181432.653629,28618.246742,147419.989802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.159507
4,473668258229864,3730753.0,81103.326087,10947.631364,8473.0,82962.0,7.705175e+06,167503.795431,10947.580825,94873.801839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99654.445592


In [20]:
#matriz=matriz.drop(['attributed_min','attributed_max,application_id_max,application_id_mean,'],axis=1)
matriz=matriz.drop('ref_hash',axis=1)
features = matriz.iloc[:,:-1]
features.head()


,hour_sum_auctions,hour_mean_auctions,hour_std_auctions,hour_min_auctions,hour_max_auctions,tiempo_desde_inicio_de_ventana_sum,tiempo_desde_inicio_de_ventana_mean,tiempo_desde_inicio_de_ventana_std,tiempo_desde_inicio_de_ventana_min,tiempo_desde_inicio_de_ventana_max,...,hour_sum_clicks,hour_mean_clicks,hour_std_clicks,hour_min_clicks,hour_max_clicks,repeticiones_sum_clicks,repeticiones_mean_clicks,repeticiones_std_clicks,repeticiones_min_clicks,repeticiones_max_clicks
0,262546.0,37506.571429,32711.731111,11871.0,84877.0,1.126550e+06,160935.721167,70802.599949,60166.331894,257677.264901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2557316.0,53277.416667,9221.332601,9576.0,58033.0,2.902939e+06,60477.901544,26439.934566,54290.380240,230833.606475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,879313.0,67639.461538,11518.707115,51449.0,80391.0,1.484119e+06,114163.007774,68866.738196,58572.637969,227124.216926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1166126.0,33317.885714,29254.773979,4460.0,86349.0,6.350143e+06,181432.653629,28618.246742,147419.989802,259149.925223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3730753.0,81103.326087,10947.631364,8473.0,82962.0,7.705175e+06,167503.795431,10947.580825,94873.801839,169362.295545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
labels = matriz.iloc[:,-1]

data_dmatrix = xgb.DMatrix(data=features,label=labels)
labels.head()

0        88.851558
1    164703.549846
2    245538.096215
3       115.159507
4     99654.445592
Name: tiempo_hasta_aparicion, dtype: float64

In [22]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = \
    train_test_split(features, labels, test_size=0.15, random_state=662)

In [23]:
#intentar usar objetive survival:cox
xg_reg = xgb.XGBRegressor(objective ='reg:linear', base_score=0.5,
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 10, n_estimators = 75,n_jobs=10,random_state=10)#RMSE: 70914

In [24]:
xg_reg.fit(train_features,train_labels)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=15, min_child_weight=1, missing=None, n_estimators=75,
       n_jobs=10, nthread=None, objective='reg:linear', random_state=10,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [25]:
predicted_labels = xg_reg.predict(test_features)

In [26]:
predicted_labels=np.asarray(predicted_labels)
predicted_labels

array([ 51646.867,  99593.43 , 217257.55 , ..., 146586.62 , 203362.75 ,
        39136.79 ], dtype=float32)

In [27]:
predicted_labels.max()

253332.62

In [28]:
rmse = np.sqrt(mean_squared_error(test_labels, predicted_labels))
print("RMSE: %f" % (rmse))

RMSE: 81386.812400


## Importancia de los features (cuantas veces aparecen en los árboles)

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [7, 11]
plt.show()

In [ ]:
features_ventana7 = pd.read_csv('../xgb/features_auctions_lucas_ventana7.csv',low_memory=False).set_index('ref_hash')
aux_ventana7 = pd.read_csv('../xgb/features_events_lucas_ventana7.csv').set_index('ref_hash')
features_ventana7=features_ventana7.join(aux_ventana7, how = 'outer', lsuffix = '_auctions', rsuffix = '_events')
aux_ventana7 = pd.read_csv('../xgb/features_installs_lucas_ventana7.csv').set_index('ref_hash')
features_ventana7=features_ventana7.join(aux_ventana7, how = 'outer', rsuffix = '_installs')
aux_ventana7 = pd.read_csv('../xgb/features_clicks_lucas_ventana7.csv').set_index('ref_hash')
features_ventana7=features_ventana7.join(aux_ventana7, how = 'outer', rsuffix = '_clicks')

labels_ventana7 = pd.read_csv('../xgb/base_target_st.csv').set_index('ref_hash')
target = labels_ventana7
features_ventana7[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']]=features_ventana7[['tiempo_desde_inicio_de_ventana_sum','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_std','tiempo_desde_inicio_de_ventana_min','tiempo_desde_inicio_de_ventana_max']].fillna(259200)#3 dias pasado a segundos

ventana7 = features_ventana7.join(labels_ventana7, how = 'right').fillna(0).reset_index().rename(index=str, columns={"device_id": "ref_hash"}) #3 dias pasado a segundos
ventana7.head()

In [ ]:
features_ventana7 = ventana7.iloc[:,:-1]
labels_ventana7 = ventana7.iloc[:,-1]

In [ ]:
data_dmatrix = xgb.DMatrix(data=features,label=labels)

In [ ]:
features_ventana7=features_ventana7.drop(['attributed_min','attributed_max','ref_hash'],axis=1)
predicted_target_labels = xg_reg.predict(features_ventana7)

In [ ]:
target.shape

In [ ]:
target['obj'] = predicted_target_labels

In [ ]:
target=target.sort_values(by='obj')

In [ ]:
target.to_csv('../xgb/target_st.csv')